<a href="https://colab.research.google.com/github/okchang95/aiffel_quest_changwoo/blob/main/Main_Quest_2/%EC%98%A5%EC%B0%BD%EC%9A%B0_Night_Ride_Image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## .. ㅠ

coco가 도대체 ㅇ떻게쓰는거죠<br>
split하고 tensor로 바꾸고싶었는데 돋체<br>
모델은 근처도 못가고 23987425시간동안 coco 쓰는법만 찾아봤습니다

# ㅠㅠ

# **gg**

<!-- task
1. Image Segmantation Model을 만들고 전체 코드와 분류 결과를 깃헙에 제출
    - 이미지데이터를 적절히 전처리, 데이터 파이프라인 설계
    - 모델은 U-Net으로 고정
    - 직접 모델 학습 코드 구현
2. 결과를 TensorBoard에 시각화
    - url 주소링크를 Readme.md 파일에 게시


A. data
B. build model
C. val, deploy

 -->


# Motercycle Night Ride (Semantic Segmentation)
---
## About Dataset
https://www.kaggle.com/datasets/sadhliroomyprime/motorcycle-night-ride-semantic-segmentation

The motorcycle ride dataset is a collection of **200 frames taken from open media available on YouTube** to enable testing for object detection and/or mobility-centered AI solutions - specifically on computer vision powered motorcycle helmets or other inventive avenues.

**Six standard classes** are used which are: <u>Undrivable, Road, Lanemark, My bike, Rider, Movable</u>

- **Movable** : denotes <u>moving objects</u> e.g. vehicles, people etc.,
- **undrivable** : denoting <u>areas where one cannot ride to</u>.
- Other classes are <u>self-explanatory inclusive</u> of **road**, **lanemark** (inclusive of reflectors), **rider**, and, of course, the **bike itself**.

We have used SuperAnnotate’s pixel editor as the tool for the semantic segmentation. It works on a raster logic as opposed to a vector one. Exporting include the COCO format. We have prepackaged the dataset inclusive of fused images.

Dataset is created by Acme AI Ltd. (www.acmeai.tech) and is #openaccess 😊 😊



### [ Contents ]
#### A. Data Laod
1. data download
2. coco to tensor
3. preprocessing

#### B. Build model
1. unet 구현
2. training, test

#### C. visualization, conclusion
1. callbacks.TensorBoard
2. conclusion

## A. Data Load

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"changwoo95","key":"02bdcff2fc5c7934c3778822f775c0d4"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sadhliroomyprime/motorcycle-night-ride-semantic-segmentation
!unzip -qq motorcycle-night-ride-semantic-segmentation.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 324M/325M [00:16<00:00, 23.7MB/s]
100% 325M/325M [00:16<00:00, 21.0MB/s]


In [61]:
data_dir = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/images'
annotation_file = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/COCO_motorcycle (pixel).json'

In [100]:
# 1. Load the COCO dataset:

import tensorflow as tf
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split

# Load the COCO dataset annotations
coco = COCO(annotation_file)

# Get image IDs and corresponding segmentation masks
image_ids = coco.getImgIds()

# 데이터를 훈련, 검증, 테스트 셋으로 나눕니다.
train_ids, val_test_ids = train_test_split(image_ids, test_size=0.3, random_state=42) # 140
val_ids, test_ids = train_test_split(val_test_ids, test_size=0.5, random_state=42)  # 30, 30

# 각 세트에 속하는 이미지의 정보를 가져옵니다.
train_images = coco.loadImgs(train_ids)
val_images = coco.loadImgs(val_ids)
test_images = coco.loadImgs(test_ids)

segmentations = coco.loadAnns(coco.getAnnIds(imgIds=image_ids, iscrowd=False))

loading annotations into memory...
Done (t=3.20s)
creating index...
index created!


In [103]:
print(len(train_images))
print(len(val_images))
print(len(test_images))


140
30
30


In [ ]:
def coco2tensor():
    pass

In [ ]:
def preprocessing():
    pass

## ...........ㅠ


In [ ]:
'''
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array
import requests
from PIL import Image

# 출력하려는 이미지의 ID를 선택합니다.
target_image_id = 123  # 원하는 이미지의 ID로 변경해야 합니다.

# 선택한 이미지를 로드합니다.
img_info = coco.loadImgs(target_image_id)[0]
img_url = img_info['coco_url']
img = Image.open(requests.get(img_url, stream=True).raw)

# 이미지를 출력합니다.
plt.imshow(img)
plt.title(f'Image ID: {target_image_id}')
plt.axis('off')  # 축을 제거하여 깔끔하게 표시합니다.
plt.show()
'''

In [64]:
# 2. Preprocess images and masks:

def preprocess_image_and_mask(image_path, segmentation):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (256, 256))  # Adjust size as needed
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.keras.utils.to_categorical(segmentation['category_id'], num_classes=80)  # Adjust num_classes
    mask = tf.image.resize(mask, (256, 256))  # Adjust size as needed

    return image, mask



In [65]:
# 3. Create a TensorFlow dataset:

dataset = tf.data.Dataset.from_tensor_slices((image_path for image_path in images))
dataset = dataset.map(lambda image_path: preprocess_image_and_mask(image_path, segmentations))
dataset = dataset.batch(32)  # Adjust batch size


image_paths = [image['file_name'] for image in images]  # Create a list of image paths
dataset = tf.data.Dataset.from_tensor_slices((image_paths, segmentations))

ValueError: ignored

In [ ]:
sample_dir = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/images'
label_dir = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/COCO_motorcycle (pixel).json'
#classes = ['Undrivable', 'Road', 'Lanemark', 'My bike', 'Rider, Movable']
classes_num = 6



In [ ]:
# 이미지 읽기
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

### COCO format?


In [50]:
from pycocotools.coco import COCO

coco_json_path = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/COCO_motorcycle (pixel).json'
coco = COCO(coco_json_path)

loading annotations into memory...
Done (t=4.06s)
creating index...
index created!


In [98]:
type(images[1])
images[1]

{'id': 2,
 'file_name': 'night ride (14).png',
 'height': 1080,
 'width': 1920,
 'license': 1}

In [51]:
# coco to tensor

import torch

def coco_to_tensor(coco_json_path, image_dir):
  """COCO 형식 이미지 데이터 세트를 텐서로 변환합니다.

  Args:
    coco_json_path: COCO 형식 이미지 데이터 세트의 JSON 파일 경로.
    image_dir: 이미지 파일이 있는 디렉토리 경로.

  Returns:
    images: 이미지 데이터 텐서.
    annotations: 태그 데이터 텐서.
  """

  # COCO 데이터 세트를 로드합니다.
  coco = COCO(coco_json_path)

  # 이미지 데이터를 로드합니다.
  images = []
  for image_id in coco.imgs.keys():
    image = coco.loadImgs(image_id)[0]
    image = torch.from_numpy(image['data']).float()
    image = image.permute(2, 0, 1)
    images.append(image)

  # 태그 데이터를 로드합니다.
  annotations = []
  for annotation in coco.loadAnns(coco.getAnnIds(imgIds=coco.imgs.keys())):
    annotation = annotation['segmentation']
    annotation = torch.from_numpy(annotation).float()
    annotations.append(annotation)

  return images, annotations


if __name__ == '__main__':
  # COCO 형식 이미지 데이터 세트의 JSON 파일 경로를 지정합니다.
  #coco_json_path = '/path/to/coco.json'

  # 이미지 파일이 있는 디렉토리 경로를 지정합니다.
  image_dir = '/content/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/images

  # 이미지 데이터 세트를 텐서로 변환합니다.
  images, annotations = coco_to_tensor(coco_json_path, image_dir)

  # 이미지 데이터 텐서를 확인합니다.
  print(images.shape)

  # 태그 데이터 텐서를 확인합니다.
  print(annotations.shape)


loading annotations into memory...
Done (t=2.59s)
creating index...
index created!


KeyError: ignored

## B. Build Model

: Unet

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def multi_unet_model(n_classes=6, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5) # copy, crop
    u6 = concatenate([u6, c4]) # crop
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6) # copy, crop
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7) # copy, crop
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8) # copy, crop
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    #model.summary()

    return model

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
'''
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
'''
model = keras.Model(inputs, outputs)